Evaluation metrics: nDCG@k and mAP@k for popularity and production baselines 
The evaluation metrics follow: https://gist.github.com/bwhite/3726239#file-rank_metrics-py

NDCG

In [1]:
import numpy as np

def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max



In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("merged_train.csv", header = None)
df_test = pd.read_csv("merged_test.csv", header = None)

In [5]:
df_test.columns = ["tracing_id", "zpid_hash", "feature1", "feature2", "feature3", "feature4", "feature5", "feature6", "feature7", "feature8","user_id_hash", "timestamp_session", "user_session_id", "order", "submit", "fav", "click"]

In [6]:
df.columns = ["tracing_id", "zpid_hash", "feature1", "feature2", "feature3", "feature4", "feature5", "feature6", "feature7", "feature8","user_id_hash", "timestamp_session", "user_session_id", "order", "submit", "fav", "click"]

In [7]:
df_test.click = df_test.click.replace({ True: 1, False: 0})
df_test.submit = df_test.submit.replace({True: 1, False: 0})
df_test.fav = df_test.fav.replace({True: 1, False: 0})

In [8]:
df.click = df.click.replace({ True: 1, False: 0})
df.submit = df.submit.replace({True: 1, False: 0})
df.fav = df.fav.replace({True: 1, False: 0})

In [9]:
df_test["weighted_score"] = 0.5*df_test["fav"] + 0.2*df_test["click"] + df_test["submit"]

In [10]:
df["weighted_score"] = 0.5*df["fav"] + 0.2*df["click"] + df["submit"]

Popularity

In [11]:
top_20_weighted = df.groupby('zpid_hash')['weighted_score'].sum().sort_values(ascending = False).head(20)

In [12]:
top_20_weighted= top_20_weighted.to_frame().reset_index()

In [13]:
top_20_weighted = top_20_weighted.drop('weighted_score', axis = 1)

In [14]:
df_test = df_test.sort_values(by = ["tracing_id", "order"], ascending = [False, False])

In [15]:
df_test_zpid = df_test.groupby("tracing_id")["zpid_hash"].apply(list).reset_index()

In [16]:
df_test

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,weighted_score
4123134,62cb67f319bef3b9f7b6c258aa7fc78c,11770ebfe6677d83e75c558c3790be42f1055f50,9738,52,6509,9581,70,10,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,1,0,0,0,0.0
4123133,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,0.2
459779,62cb67f1265d7ed45bd56d5bf981af4a,5afc66bbc93cea5309c01b8444aa39997aa0fa69,8611,77,1001,555,1034,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,25,0,0,0,0.0
459773,62cb67f1265d7ed45bd56d5bf981af4a,43d757c42467a59829f164c8fede15972ac8a67b,8611,77,1001,555,27,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,24,0,0,0,0.0
459772,62cb67f1265d7ed45bd56d5bf981af4a,497a9a9c80f5ef8975d31e269859be255422b3fb,8611,1903,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,23,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103648,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,0.0
3103644,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,0.2
3103650,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,0.0
3103612,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,0.0


In [17]:
df_test_zpid

,tracing_id,zpid_hash
0,62ca193dc36929bb30f091105b8e5bd1,"[2bbb95c9cdd5dcff958e4519d6466edeb3c0ff00, 7c9..."
1,62ca193dcfc19c8d05fb5d9ef28d608e,"[5f92e292b1263da801825dfa17ca0dae36502b34, 1a0..."
2,62ca193dd0a6155c1595dd3ab61852ae,"[2cbf84dcec7a509f381728c0bf5d452130e8e45, 6eb6..."
3,62ca193dd4eff6309eb1cd23e3d0df62,"[0863149f92b7f0b9c7cf836e9ef0503c68c591ca, 625..."
4,62ca193dd968bf1881b0fdf86252a0f1,"[b2f482aebfe43c51abb6ed59ebcff8c1d7acbaf8, 4e8..."
...,...,...
393507,62cb67ebcdcd215bd2b3713c1c99c152,"[60febee233c5ba27053f1115d53f69b1309342a5, bc4..."
393508,62cb67eeb312a541990e794692ce2a4a,"[68d5886d760c4414c8a4711e6956ef30924240f5, 4fb..."
393509,62cb67ef76d830cf4b0c83d553cd0043,"[775691a2259e5f80bf5243cad40c63b2539d311, 3c14..."
393510,62cb67f1265d7ed45bd56d5bf981af4a,"[5afc66bbc93cea5309c01b8444aa39997aa0fa69, 43d..."


In [18]:
top_20_weighted

,zpid_hash
0,1d5c788c75fd7153119ee2e9ff0e411ebd4614e
1,5b716b926169de32b4cbc2b5099966e564dc45e1
2,5fc510c9b80f64db0b5eecd102e7a833ef9d6614
3,7d8183eabe736ec9f38811a487c7218d6f5d1de1
4,9c6d2e62fa2616ecd4787ca7a2dbff0ff3a76244
5,06fc8b54fb7138cac722981b418b276704f3ce7d
6,6e6eb409a354e81ab65b5a12a67d8764af770f54
7,831955e4693a411d80722b0b031865eb0da699d0
8,e3d9c5a77a23a779dac659994d4df44f2ae9b14d
9,c2bfc97369928a24c2718e7306cd97e25dcfd72f


In [19]:
df_test_zpid["r"] = [[0 for i in range(20)] for i in range(len(df_test_zpid))]

In [20]:
def in_dict(i, k):
    property_id = top_20_weighted["zpid_hash"][k]
    if property_id in df_test_zpid["zpid_hash"][i]:
        return (df_test_zpid["zpid_hash"][i]).index(property_id) + 1
    else:
        return 0

In [21]:
df_test_zpid["r"] = [[in_dict(i, k) for k in range(20)] for i in range(len(df_test_zpid))]

In [22]:
df_test_zpid

,tracing_id,zpid_hash,r
0,62ca193dc36929bb30f091105b8e5bd1,"[2bbb95c9cdd5dcff958e4519d6466edeb3c0ff00, 7c9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,62ca193dcfc19c8d05fb5d9ef28d608e,"[5f92e292b1263da801825dfa17ca0dae36502b34, 1a0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,62ca193dd0a6155c1595dd3ab61852ae,"[2cbf84dcec7a509f381728c0bf5d452130e8e45, 6eb6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,62ca193dd4eff6309eb1cd23e3d0df62,"[0863149f92b7f0b9c7cf836e9ef0503c68c591ca, 625...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,62ca193dd968bf1881b0fdf86252a0f1,"[b2f482aebfe43c51abb6ed59ebcff8c1d7acbaf8, 4e8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
393507,62cb67ebcdcd215bd2b3713c1c99c152,"[60febee233c5ba27053f1115d53f69b1309342a5, bc4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
393508,62cb67eeb312a541990e794692ce2a4a,"[68d5886d760c4414c8a4711e6956ef30924240f5, 4fb...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
393509,62cb67ef76d830cf4b0c83d553cd0043,"[775691a2259e5f80bf5243cad40c63b2539d311, 3c14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
393510,62cb67f1265d7ed45bd56d5bf981af4a,"[5afc66bbc93cea5309c01b8444aa39997aa0fa69, 43d...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
total = 0
for i in range(len(df_test_zpid)):
    total = total + ndcg_at_k(df_test_zpid["r"][i],20, method=1)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + ndcg_at_k(df_test_zpid["r"][i],10, method=1)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + ndcg_at_k(df_test_zpid["r"][i],5, method=1)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + ndcg_at_k(df_test_zpid["r"][i],2, method=1)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + ndcg_at_k(df_test_zpid["r"][i],1, method=1)

print(total/len(df_test_zpid))


0.007368394333497872
0.0053896006893803085
0.0036305857628166007
0.0026004526329199023
0.0016910172366489382


Production

In [32]:
df_test = df_test.sort_values(by = ["tracing_id", "order"], ascending = [False, False])

In [33]:
df_test_2 =  df_test[df_test["weighted_score"] > 0 ]

In [34]:
df_test_2["relevance"] = df_test_2.groupby(["tracing_id"])["weighted_score"].rank(method='first', ascending = False)

<ipython-input-34-fd98722aabd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2["relevance"] = df_test_2.groupby(["tracing_id"])["weighted_score"].rank(method='first', ascending = False)


In [35]:
df_test_2 = df_test_2[["tracing_id", "zpid_hash", "relevance"]]

In [36]:
df_test

,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,weighted_score
4123134,62cb67f319bef3b9f7b6c258aa7fc78c,11770ebfe6677d83e75c558c3790be42f1055f50,9738,52,6509,9581,70,10,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,1,0,0,0,0.0
4123133,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,0.2
459779,62cb67f1265d7ed45bd56d5bf981af4a,5afc66bbc93cea5309c01b8444aa39997aa0fa69,8611,77,1001,555,1034,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,25,0,0,0,0.0
459773,62cb67f1265d7ed45bd56d5bf981af4a,43d757c42467a59829f164c8fede15972ac8a67b,8611,77,1001,555,27,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,24,0,0,0,0.0
459772,62cb67f1265d7ed45bd56d5bf981af4a,497a9a9c80f5ef8975d31e269859be255422b3fb,8611,1903,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,23,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103648,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,0.0
3103644,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,0.2
3103650,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,0.0
3103612,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,0.0


In [37]:
df_test_2

,tracing_id,zpid_hash,relevance
4123133,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,1.0
459762,62cb67f1265d7ed45bd56d5bf981af4a,486ed025904d4328370388e4e6b967b3cdbd0fed,1.0
2335194,62cb67ef76d830cf4b0c83d553cd0043,bbfc79ec1a31a6f3cc62e119fbe791b4898a644a,1.0
197791,62cb67eeb312a541990e794692ce2a4a,1536719c1d09878082809afbad2eed6291e136,1.0
2142652,62cb67eab13051ce4cfff33a86fb243f,6f9c0dca93f0edf551bf1092176454da020f4059,1.0
...,...,...,...
3932249,62ca193dcfc19c8d05fb5d9ef28d608e,016c7f46d6bab115af55401145388b8666fc4496,1.0
3103624,62ca193dc36929bb30f091105b8e5bd1,051cbec0834e7e4838f9590c9a352c31c8efafe7,1.0
3103651,62ca193dc36929bb30f091105b8e5bd1,f0b2717e076ea055e72aa9b0d3bd7df9a7f10377,2.0
3103633,62ca193dc36929bb30f091105b8e5bd1,5db96e84277e533ca6d2c55fc8408c8fc3a0ecc0,3.0


In [38]:
df_test_prod = df_test_2.merge(df_test, on= ["tracing_id", "zpid_hash"], how = "right") 

In [39]:
df_test_prod["relevance"] = df_test_prod["relevance"].fillna(0)

In [40]:
df_test_prod 

,tracing_id,zpid_hash,relevance,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,weighted_score
0,62cb67f319bef3b9f7b6c258aa7fc78c,11770ebfe6677d83e75c558c3790be42f1055f50,0.0,9738,52,6509,9581,70,10,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,1,0,0,0,0.0
1,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,1.0,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,0.2
2,62cb67f1265d7ed45bd56d5bf981af4a,5afc66bbc93cea5309c01b8444aa39997aa0fa69,0.0,8611,77,1001,555,1034,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,25,0,0,0,0.0
3,62cb67f1265d7ed45bd56d5bf981af4a,43d757c42467a59829f164c8fede15972ac8a67b,0.0,8611,77,1001,555,27,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,24,0,0,0,0.0
4,62cb67f1265d7ed45bd56d5bf981af4a,497a9a9c80f5ef8975d31e269859be255422b3fb,0.0,8611,1903,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,23,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576042,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,0.0,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,0.0
12576043,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,4.0,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,0.2
12576044,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,0.0,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,0.0
12576045,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,0.0,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,0.0


In [41]:
df_test_prod = df_test_prod.sort_values(by =["tracing_id","order"], ascending = [True, True])

In [42]:
df_test_prod

,tracing_id,zpid_hash,relevance,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,user_id_hash,timestamp_session,user_session_id,order,submit,fav,click,weighted_score
12576046,62ca193dc36929bb30f091105b8e5bd1,8547e3c3f8e4987b028bdb26bdcbbad4f594b5ea,0.0,9418,6594,8848,9069,1212,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,0,0,0,0,0.0
12576045,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,0.0,9418,475,8848,9069,11,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,1,0,0,0,0.0
12576044,62ca193dc36929bb30f091105b8e5bd1,eacffeb4d791ae64fbb8959d445cb414b205df7c,0.0,9418,6594,8848,9069,2111,23,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,2,0,0,0,0.0
12576043,62ca193dc36929bb30f091105b8e5bd1,b79a7a68c1d71c9305317402023ddd86b1289fe4,4.0,9418,475,8848,9069,3507,1126,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,3,0,0,1,0.2
12576042,62ca193dc36929bb30f091105b8e5bd1,5260c39a095eb64846e7c7d6b22c076da5268e2b,0.0,9418,6594,8848,9069,408,127,1395,1423,27ff6af7087d26ab7749f0c831569954b6d155f7,2022-07-10 00:11:42.058,8b5b9a5615874fba9f7c3d2d60a7eb61_2022_07_10_00_1,4,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,62cb67f1265d7ed45bd56d5bf981af4a,497a9a9c80f5ef8975d31e269859be255422b3fb,0.0,8611,1903,1001,7777,158,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,23,0,0,0,0.0
3,62cb67f1265d7ed45bd56d5bf981af4a,43d757c42467a59829f164c8fede15972ac8a67b,0.0,8611,77,1001,555,27,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,24,0,0,0,0.0
2,62cb67f1265d7ed45bd56d5bf981af4a,5afc66bbc93cea5309c01b8444aa39997aa0fa69,0.0,8611,77,1001,555,1034,55,1325,1001,9d9f8a31964ca0ca47e3fee4345a2a0af6045f1c,2022-07-10 23:59:45.865,f76bb1fba1cd4ac19553f6c2d2a04630_2022_07_10_23_1,25,0,0,0,0.0
1,62cb67f319bef3b9f7b6c258aa7fc78c,b5cf95f407257181efde17f5c27e637f171102e2,1.0,9738,8750,6509,9581,610,5877,6398,6398,806275b220719de60849a5aeab92f6ee56f402ab,2022-07-10 23:59:47.119,9df1e8c1f1b742518d157119b54490d3_2022_07_10_23_1,0,0,0,1,0.2


In [43]:
df_test_prod = df_test_prod.groupby("tracing_id")["relevance"].apply(list).reset_index()

In [44]:
df_test_prod

,tracing_id,relevance
0,62ca193dc36929bb30f091105b8e5bd1,"[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 2.0, ..."
1,62ca193dcfc19c8d05fb5d9ef28d608e,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,62ca193dd0a6155c1595dd3ab61852ae,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,62ca193dd4eff6309eb1cd23e3d0df62,"[2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,62ca193dd968bf1881b0fdf86252a0f1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
393507,62cb67ebcdcd215bd2b3713c1c99c152,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
393508,62cb67eeb312a541990e794692ce2a4a,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
393509,62cb67ef76d830cf4b0c83d553cd0043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
393510,62cb67f1265d7ed45bd56d5bf981af4a,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [45]:
total = 0
for i in range(len(df_test_prod)):
    total = total + ndcg_at_k(df_test_prod["relevance"][i],20, method=1)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + ndcg_at_k(df_test_prod["relevance"][i],10, method=1)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + ndcg_at_k(df_test_prod["relevance"][i],5, method=1)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + ndcg_at_k(df_test_prod["relevance"][i],2, method=1)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + ndcg_at_k(df_test_prod["relevance"][i],1, method=1)

print(total/len(df_test_prod))


0.47365889953056806
0.4269447320790435
0.3713940598581317
0.3038335862134293
0.26661305538442587


MAP

In [25]:
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    >>> r = [0, 0, 1]
    >>> precision_at_k(r, 1)
    0.0
    >>> precision_at_k(r, 2)
    0.0
    >>> precision_at_k(r, 3)
    0.33333333333333331
    >>> precision_at_k(r, 4)
    Traceback (most recent call last):
        File "<stdin>", line 1, in ?
    ValueError: Relevance score length < k
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)


def average_precision(r, k):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).
    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    #print(r)
    
    if k not in range(len(r)): 
        padding = [False for i in range(len(r), k+1)]
        #print(padding)
        r = np.concatenate((r, padding), axis =0)
    
    out = [precision_at_k(r, i+1) for i in range(k)]
    
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs, k):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r, k) for r in rs])



Popularity

In [55]:
total = 0
for i in range(len(df_test_zpid)):
    total = total + mean_average_precision([df_test_zpid["r"][i]],20)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + mean_average_precision([df_test_zpid["r"][i]],10)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + mean_average_precision([df_test_zpid["r"][i]],5)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + mean_average_precision([df_test_zpid["r"][i]],2)

print(total/len(df_test_zpid))

total = 0
for i in range(len(df_test_zpid)):
    total = total + mean_average_precision([df_test_zpid["r"][i]],1)

print(total/len(df_test_zpid))


0.0015514768850202658
0.0015592721482122305
0.0015603420819017733
0.0016803807761897986
0.0017331110614161702


Production

In [56]:
total = 0
for i in range(len(df_test_prod)):
    total = total + mean_average_precision([df_test_prod["relevance"][i]],20)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + mean_average_precision([df_test_prod["relevance"][i]],10)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + mean_average_precision([df_test_prod["relevance"][i]],5)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + mean_average_precision([df_test_prod["relevance"][i]],2)

print(total/len(df_test_prod))

total = 0
for i in range(len(df_test_prod)):
    total = total + mean_average_precision([df_test_prod["relevance"][i]],1)

print(total/len(df_test_prod))


0.13893502800847246
0.17579843373026258
0.21166606524468973
0.2500825896033666
0.2679664152554433


DCN

In [89]:
dcn_pred = pd.read_csv("pred_dcn_newest.csv")

In [13]:
dcn_pred = dcn_pred.sort_values(by = ["tracing_id", "weighted"])

In [14]:
dcn_pred 

,Unnamed: 0,tracing_id,zpid_hash,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,weighted,pred,weight_rank
3103611,3103611,62ca193dc36929bb30f091105b8e5bd1,004b8596022713dff74dc9df1c0d4e936b82022a,9418,475,8848,9069,11,127,1395,1423,0.0,0.014543,5.0
3103612,3103612,62ca193dc36929bb30f091105b8e5bd1,1969f71423842e14ecc5ef487ea6732d8b9dc6b,9418,475,8848,9069,177,143,1395,1423,0.0,0.009490,6.0
3103613,3103613,62ca193dc36929bb30f091105b8e5bd1,c1716e3d8ec13e3801991cd9ab089d0fc6d2314d,9418,6594,8848,232,11,23,1395,1423,0.0,0.003298,7.0
3103614,3103614,62ca193dc36929bb30f091105b8e5bd1,130a4afc22d8b5e9dd21dbbae2517a5fdc033f2,9418,475,8848,9069,11,23,1395,1423,0.0,0.003855,8.0
3103615,3103615,62ca193dc36929bb30f091105b8e5bd1,8547e3c3f8e4987b028bdb26bdcbbad4f594b5ea,9418,6594,8848,9069,1212,1126,1395,1423,0.0,0.180156,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459782,459782,62cb67f1265d7ed45bd56d5bf981af4a,c6f8f70e3a11180830f513973d6e9f7fb93cdd68,8611,1903,1001,7777,174,525,1325,1001,0.0,0.013573,25.0
459783,459783,62cb67f1265d7ed45bd56d5bf981af4a,e5d078c1477513070d05947b09ccb7c90d76479b,8611,62,1001,7777,810,193,1325,1001,0.0,0.128040,26.0
459761,459761,62cb67f1265d7ed45bd56d5bf981af4a,486ed025904d4328370388e4e6b967b3cdbd0fed,8611,3698,1001,7777,1383,55,1325,1001,0.2,0.045285,1.0
4123133,4123133,62cb67f319bef3b9f7b6c258aa7fc78c,11770ebfe6677d83e75c558c3790be42f1055f50,9738,52,6509,9581,70,10,6398,6398,0.0,0.006989,2.0


In [15]:
dcn_pred["pred_rank"] =  dcn_pred.groupby(["tracing_id"])["pred"].rank(method='first', ascending = False)

In [18]:
dcn_pred_r = dcn_pred.groupby(["tracing_id"])["pred_rank"].apply(list).reset_index()

In [20]:
dcn = dcn_pred_r

NDCG

In [23]:
total = 0
for i in range(len(dcn)):
    total = total + ndcg_at_k(dcn["pred_rank"][i],20, method=1)

print(total/len(dcn))

total = 0
for i in range(len(dcn)):
    total = total + ndcg_at_k(dcn["pred_rank"][i],10, method=1)

print(total/len(dcn))

total = 0
for i in range(len(dcn)):
    total = total + ndcg_at_k(dcn["pred_rank"][i],5, method=1)

print(total/len(dcn))

total = 0
for i in range(len(dcn)):
    total = total + ndcg_at_k(dcn["pred_rank"][i],2, method=1)

print(total/len(dcn))

total = 0
for i in range(len(dcn)):
    total = total + ndcg_at_k(dcn["pred_rank"][i],1, method=1)

print(total/len(dcn))


0.7058864149960347
0.6347509801038754
0.5942886396092919
0.5643245610248097
0.5508570073260194
